In [0]:
#importing pyspark
import pyspark

from pyspark.sql import SparkSession

# Creating a SparkSession
spark = SparkSession.builder.getOrCreate()


In [0]:
#File moved to the required path
#dbutils.fs.mv("dbfs:/FileStore/shared_uploads/amanjainworkemail@gmail.com/skill2vec_50K_csv-1.gz",\
               #"/FileStore/tables/skill2vec_50K.csv.gz")
#dbutils.fs.mv("dbfs:/FileStore/shared_uploads/amanjainworkemail@gmail.com/Technology_Skills.txt",\ 
               #"/FileStore/tables/Technology_Skills.txt")

In [0]:
# List all the files uploaded in the databrick directory
files = dbutils.fs.ls("/FileStore/tables")

# Filter files to include only those files which are ending with .csv or .txt extensions
file_names = [file.name for file in files if file.name.endswith((".txt", ".gz"))]

# Printing the names of the filtered files
for file_name in file_names:
    print(file_name)

Technology_Skills.txt
skill2vec_50K.csv.gz


In [0]:
# Defining a function to read all the files in the dictionary
def read_files(file_dict):
    """
    Reads multiple files and returns a dictionary of PySpark DataFrame

    Parameters:
        file_dict (dict): A dictionary where the keys are the file names and the values are the file paths.

    Returns:
        A dictionary where the keys are the file names and the values are PySpark DataFrames.
    """
    
    # Create an empty dictionary to store the file data
    file_data = {}

    # Iterate over each file in the file dictionary
    for file_name, file_path in file_dict.items():
        # Check the file extension and read the file accordingly
        if file_path.endswith(".csv") or file_path.endswith(".csv.gz"):
            df = spark.read.format("csv").option("header", "false").load(file_path)
        elif file_path.endswith(".txt"):
            df = spark.read.csv(file_path, sep="\t", header=True)
        else:
            raise ValueError("File format not supported: {}".format(file_path))

        # Store the DataFrame in the file_data dictionary
        file_data[file_name] = df

    # Return the dictionary containing the file data
    return file_data

# Define the file dictionary with file names and paths
file_dict = {
    "skill": "/FileStore/tables/skill2vec_50K.csv.gz",
    "o_net": "/FileStore/tables/Technology_Skills.txt"
}

# Read the files and obtain the dictionary of PySpark DataFrames
file_data = read_files(file_dict)

# Iterate over the file_data dictionary and print the keys (file names)
for key, value in file_data.items():
    print(key)

# Extract and print the individual PySpark DataFrames
main_dataframe = file_data['skill']
o_net = file_data['o_net']

skill
o_net


In [0]:
# Print the schema of the DataFrames
main_dataframe.printSchema()
o_net.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [0]:
from pyspark.sql.functions import col

# Defining a function to rename columns of the dataframe
def rename_columns(df, prefix):
    """
    Renames columns of a PySpark DataFrame with a given prefix.

    Parameters:
        df: PySpark DataFrame to modify
        prefix (str): Prefix to add to each column name.

    Returns:
        A new PySpark DataFrame with modified column names.
    """
    # Generate a list of new column names by adding the prefix to each existing column name
    column_names = [prefix + str(i+1) for i in range(len(df.columns))]
    
    # Select the columns from the DataFrame and alias them with the new column names
    # using the `col` function from PySpark
    modified_df = df.select([col(c).alias(column_names[i]) for i, c in enumerate(df.columns)])
    
    return modified_df

# Call the `rename_columns` function to rename columns of the `skill_10k` DataFrame
# with the prefix "column"
main_dataframe = rename_columns(main_dataframe, "column")

# Print the schema of the modified DataFrame
main_dataframe.printSchema()

root
 |-- column1: string (nullable = true)
 |-- column2: string (nullable = true)
 |-- column3: string (nullable = true)
 |-- column4: string (nullable = true)
 |-- column5: string (nullable = true)
 |-- column6: string (nullable = true)
 |-- column7: string (nullable = true)
 |-- column8: string (nullable = true)
 |-- column9: string (nullable = true)
 |-- column10: string (nullable = true)
 |-- column11: string (nullable = true)
 |-- column12: string (nullable = true)
 |-- column13: string (nullable = true)
 |-- column14: string (nullable = true)
 |-- column15: string (nullable = true)
 |-- column16: string (nullable = true)
 |-- column17: string (nullable = true)
 |-- column18: string (nullable = true)
 |-- column19: string (nullable = true)
 |-- column20: string (nullable = true)
 |-- column21: string (nullable = true)
 |-- column22: string (nullable = true)
 |-- column23: string (nullable = true)
 |-- column24: string (nullable = true)
 |-- column25: string (nullable = true)
 |--

In [0]:
from pyspark.sql.functions import concat_ws, count

# Defining a function to concatenate the skills into one column
def concatenate_columns(df):
    """
    Concatenates all columns in a PySpark DataFrame except the first column.

    Parameters:
        dataframe: PySpark DataFrame to modify

    Returns:
        A new PySpark DataFrame with concatenated column.
    """
    # Get all column names of the DataFrame
    columns = df.columns

    # Concatenate all skills columns into a single column
    df = df.withColumn("skills", concat_ws(",", *[col(c) for c in columns[1:]]))

    # Rename the first column to 'JD'
    df = df.withColumnRenamed(columns[0], "JD")
    
    # Change datatype of 'JD' column to integer
    df = df.withColumn("JD", col("JD").cast("integer"))

    # Select only first column 'JD' and concenated column 'skills'
    df = df.select("JD", "skills")

    #returning the dataframe to the funtion call
    return df

# Call the `concatenate_columns` function to concatenate all columns in the `skill_50k` DataFrame
# except the first column i.e job identifier
main_dataframe = concatenate_columns(main_dataframe)

# Print the schema of the modified DataFrame
main_dataframe.printSchema()

root
 |-- JD: integer (nullable = true)
 |-- skills: string (nullable = false)



In [0]:
# Defining a function to count null and not null values
def count_null_not_null(df, column_name):
    """
    Counts the number of null and non-null in a column of a 
    PySpark DataFrame.

    Parameters:
        df: PySpark DataFrame to modify
        column_name: The name of the column to count the null and non-null values 

    Returns:
        A tuple with the number of null values and non-null values,
        respectively.
    """
    # Count the number of null values in the specified column
    null_count = df.filter(col(column_name).isNull()).count()

    # Count the number of non-null values in the specified column
    not_null_count = df.filter(col(column_name).isNotNull()).count()
    
    # Count the number of distinct values in the specified column
    distinct_values = df.select(column_name).distinct().count()

    return distinct_values, null_count, not_null_count

# Call the `count_null_not_null` function to count null and non-null values
# in the "JD" column of the `skill_10k_02` DataFrame
distinct_values, null_count, not_null_count = count_null_not_null(main_dataframe, "JD")


# Print the results
print("Number of null values in job description column:", null_count)
print("Number of non-null values in job description column:", not_null_count)
print("Number of distinct_values in job description column:",distinct_values)

Number of null values in job description column: 0
Number of non-null values in job description column: 50000
Number of distinct_values in job description column: 50000


In [0]:
from pyspark.sql.functions import split, explode

# Split the skills column into individual skills
split_df = main_dataframe.withColumn("skills", split(main_dataframe["skills"], ","))

# Explode the skills column to create a new row for each skill
exploded_df = split_df.select("JD", explode(split_df.skills).alias("skill"))

#show exploaded value of exploaded_Df dataframe
exploded_df.show()

+------+--------------------+
|    JD|               skill|
+------+--------------------+
|125720|        HR Executive|
|125720|           screening|
|125720|           selection|
|125720|           Interview|
|125720|                  HR|
|125720|           Recruiter|
|125720|        IT Recruiter|
|125720|            Sourcing|
|125720|recruitment execu...|
|125720|          onboarding|
|125720|      IT Recruitment|
|112708|     Special Teacher|
|112708|            Teaching|
|112708|           Education|
|115226|          consulting|
|115226|             fresher|
|115226|         IT helpdesk|
|115226|Techincal Trouble...|
|115226| international voice|
|115226|   international BPO|
+------+--------------------+
only showing top 20 rows



In [0]:
# QUESTION 1

# Defining a function to count distinct job desctiption
def count_distinct_jd(df, column_name):
    """
    Counts the number of distinct job descriptions in a PySpark DataFrame and prints the result.

    Parameters:
        df: PySpark DataFrame to modify
        column_name: The name of the column to count the distinct values 

    Returns:
        None
    """
    # Count the number of distinct job descriptions by selecting the "JD" column,
    # applying the `distinct` function, and then counting the result
    num_distinct = df.select(column_name).distinct().count()

    # Print the formatted count of distinct job descriptions
    print('{0:,.0f}'.format(num_distinct))

print("50K")
# count_distinct_jd funtion call to count the distinch job description
count_distinct_jd(main_dataframe, "JD")


50K
50,000


In [0]:
# QUESTION 2

from pyspark.sql.functions import count

# Defining a function to count frequency of individual skills
def count_skills(df, column_name, num_top_skills):
    """
    Counts the frequency of individual skills in a PySpark DataFrame and returns the top n most frequent skills.

    Parameters:
        df: PySpark DataFrame to modify
        column_name (str): The name of the column in the DataFrame that contains the skills
        num_top_skills (int): The number of top skills to return

    Returns:
        None
    """
    
    #dropping duplicate values
    exploded_df1 = exploded_df.drop_duplicates()
    
    # Group the DataFrame by skill and count the frequency of each skill
    skill_count = exploded_df1.groupBy("skill").agg(count("*").alias("Frequency"))

    # Get the top N skills based on frequency
    top_skills = skill_count.orderBy("Frequency", ascending=False).limit(num_top_skills)
    
    # display the top skills without truncating the output
    top_skills.show(truncate = False)
    
# calling funtion count_skills
count_skills(main_dataframe, "skills",10)


+--------------------+---------+
|skill               |Frequency|
+--------------------+---------+
|Java                |1911     |
|Javascript          |1770     |
|Sales               |1705     |
|Business Development|1545     |
|Web Technologies    |1313     |
|Communication Skills|1305     |
|development         |1238     |
|Marketing           |1184     |
|Finance             |1078     |
|HTML                |1067     |
+--------------------+---------+



In [0]:
# 1. FURTHER EXPLORATION OF QUESTION 2 

from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import count

# Instantiate a StopWordsRemover object and specify the input and output columns:
stopwords = StopWordsRemover(inputCol="skills", outputCol="skill")

# Apply the stop words removal transformation to the split_Df DataFrame:
df_filtered = stopwords.transform(split_df)

# Explode the skill_list column to create a new row for each skill
new_exploaded = df_filtered.select("JD", explode(df_filtered.skill).alias("skills"))

#T otal number of skills before removing stopwords
print("Total number of skills before removing stopwords", exploded_df.count())

# Total number of skills after removing stopwords
print("Total number of skills after removing stopwords", new_exploaded.count())


Total number of skills before removing stopwords 463908
Total number of skills after removing stopwords 462903


In [0]:
# 2. FURTHER EXPLORATION OF QUESTION 2 

from pyspark.sql.functions import count

# Defining a function to count frequency of individual skills
def count_skills(df, column_name, num_top_skills):
    """
    Counts the frequency of individual skills in a PySpark DataFrame and returns the top n 
    most frequent skills.

    Parameters:
        df: PySpark DataFrame to modify
        column_name (str): The name of the column in the DataFrame that contains the skills
        num_top_skills (int): The number of top skills to return

    Returns:
        None
    """
    
    
    # Dropping the skills which have numbers and not words
    duplicate1 = new_exploaded.groupBy("skills").count().filter(col("skills") > 1)

    # Get the values to remove as a list
    values_to_remove = [row[0] for row in duplicate1.select("skills").collect()]
    
    # Remove values from the DataFrame that are in values_to_remove
    exploded_df1 = new_exploaded.filter(~col("skills").isin(values_to_remove))
    
    #dropping duplicate values
    exploded_df1 = exploded_df1.drop_duplicates()
    
    # Group the DataFrame by skill and count the frequency of each skill
    skill_count = exploded_df1.groupBy("skills").agg(count("*").alias("Frequency"))

    # Get the top N skills based on frequency
    top_skills = skill_count.orderBy("Frequency", ascending=False).limit(num_top_skills)
    
    # display the numeric values 
    display("numeric values in the skills column are", values_to_remove)
    
    # display the top skills without truncating the output
    display(top_skills)
    
# calling funtion count_skills
count_skills(main_dataframe, "skills",10)


'numeric values in the skills column are'['50001',
 '7',
 '2016',
 '2012',
 '138',
 '1099',
 '3',
 '8',
 '2017',
 '2014',
 '8300',
 '2013',
 '4.5',
 '16',
 '61850',
 '2000',
 '4300',
 '9001',
 '573',
 '826',
 '825',
 '300',
 '5',
 '401',
 '13485',
 '286',
 '3.5',
 '18001',
 '1040',
 '201',
 '90',
 '429',
 '2105',
 '2008',
 '27001',
 '156',
 '22000',
 '4.0',
 '9000',
 '8051',
 '108',
 '60870',
 '50000',
 '24',
 '26262',
 '10',
 '802.11',
 '2015',
 '510',
 '400',
 '12',
 '+2',
 '508',
 '661',
 '2010',
 '2003',
 '14001',
 '820',
 '820.2']

skills,Frequency
Java,1911
Javascript,1770
Sales,1705
Business Development,1545
Web Technologies,1313
Communication Skills,1305
development,1238
Marketing,1184
Finance,1078
HTML,1067


In [0]:
 # QUESTION 3

from pyspark.sql.functions import size, array_distinct

# Defining a function to count frequency of individual skills and job description
def frequent_numbers_of_skills(df, num_top_freq_skills):
    
    """
    Counts the frequency of individual skills and job description 
    in a PySpark DataFrame

    Parameters:
        df: PySpark DataFrame to modify
        num_top_freq_skills (int): The number of top skills to return

    Returns:
        None
    """
    #removing duplicates
    split_df1=split_df.drop_duplicates()

    # Count the number of words in the skills column
    df_num_skills = split_df1.withColumn("num skills", size(array_distinct("skills")))

    # Group the DataFrame by the number of skills and count the number of job descriptions for each group
    df_skill_counts = df_num_skills.groupBy("num skills").count()

    # Sort the DataFrame by the count in descending order and select the top 5 rows
    df_top_skills = df_skill_counts.sort("count", ascending=False).limit(num_top_freq_skills)

    # Display the results
    df_top_skills.show(truncate=False)
    
# calling funtion frequent_numbers_of_skills
frequent_numbers_of_skills(main_dataframe, 5)


+----------+-----+
|num skills|count|
+----------+-----+
|10        |10477|
|5         |3432 |
|6         |3405 |
|1         |3386 |
|7         |3345 |
+----------+-----+



In [0]:
# FURTHER EXPLORATION OF QUESTION 3

from pyspark.sql.functions import size, array_distinct
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType


# Define a UDF to remove numeric values from an array
def remove_numeric_values(arr):
    return [val for val in arr if not (isinstance(val, int) or isinstance(val, float))]
    
    
# Defining a function to count frequency of individual skills and job description
def frequent_numbers_of_skills(df, num_top_freq_skills):
    
    """
    Counts the frequency of individual skills and job description 
    in a PySpark DataFrame

    Parameters:
        df: PySpark DataFrame to modify
        num_top_freq_skills (int): The number of top skills to return

    Returns:
        None
    """
    #removing duplicates
    split_df1=df_filtered.drop_duplicates()
    
    # Define a UDF for the remove_numeric_values function
    remove_numeric_values_udf = udf(remove_numeric_values, ArrayType(StringType()))

    # Remove numeric values from the "skills" column using the UDF
    split_df1 = split_df.withColumn("skills", remove_numeric_values_udf(col("skills")))

    # Count the number of words in the skills column
    df_num_skills = split_df1.withColumn("num skills", size(array_distinct("skills")))

    # Group the DataFrame by the number of skills and count the number of job descriptions for each group
    df_skill_counts = df_num_skills.groupBy("num skills").count()

    # Sort the DataFrame by the count in descending order and select the top 5 rows
    df_top_skills = df_skill_counts.sort("count", ascending=False).limit(num_top_freq_skills)

    # Display the results
    df_top_skills.show(truncate=False)
    
# calling funtion frequent_numbers_of_skills
frequent_numbers_of_skills(main_dataframe, 5)


+----------+-----+
|num skills|count|
+----------+-----+
|10        |10477|
|5         |3432 |
|6         |3405 |
|1         |3386 |
|7         |3345 |
+----------+-----+



In [0]:
# QUESTION 4
from pyspark.sql.functions import lower

# Define a function convert column in dataframe to lowercase
def convert_to_lower(df, column):
    """
    convert a column in a datframe to lowercase
    Parameters:
        df: PySpark DataFrame to modify
        column: column name to convert to lowercase
    Returns:
        dataframe and column name which is converted to lowercase
    """
    return df.withColumn(column, lower(df[column]))

# Define a function count distinct lowercase skills 
def distinct_skills_lowercase(df, num_top_skills):
    """
    Counts the frequency of individual skills in a PySpark DataFrame
    Parameters:
        df: PySpark DataFrame to modify
        num_top_skills (int): The number of top skills to return
    Returns:
        None
    """
    #remove duplicates values
    exploded_df1=exploded_df.drop_duplicates()
    
    # Convert the exploded column "skills" to lowercase
    exploded_df_lower = convert_to_lower(exploded_df1,"skill")
    
    # Group the DataFrame by skill and count the frequency of each skill
    exploded_df_lower = exploded_df_lower.groupBy("skill").agg(count("skill").alias("Frequency"))

    # Display the top 10 distinct skills in lowercase and their frequencies
    exploded_df_lower.orderBy("Frequency", ascending=False).show(num_top_skills, truncate=False)
    
# Funtion Call
distinct_skills_lowercase(main_dataframe, 10)


+--------------------+---------+
|skill               |Frequency|
+--------------------+---------+
|java                |2759     |
|javascript          |2738     |
|sales               |2680     |
|business development|2108     |
|marketing           |1809     |
|sql                 |1564     |
|jquery              |1547     |
|html                |1539     |
|communication skills|1537     |
|bpo                 |1530     |
+--------------------+---------+
only showing top 10 rows



In [0]:
#1. FURTHER EXPLORATION QUESTION 4

from pyspark.sql.functions import lower

# Define a function convert column in dataframe to lowercase
def convert_to_lower(df, column):
    """
    convert a column in a datframe to lowercase
    Parameters:
        df: PySpark DataFrame to modify
        column: column name to convert to lowercase
    Returns:
        dataframe and column name which is converted to lowercase
    """
    return df.withColumn(column, lower(df[column]))

# Define a function count distinct lowercase skills 
def distinct_skills_lowercase(df, num_top_skills):
    """
    Counts the frequency of individual skills in a PySpark DataFrame
    Parameters:
        df: PySpark DataFrame to modify
        num_top_skills (int): The number of top skills to return
    Returns:
        None
    """
    #remove duplicates values
    exploded_df1 = new_exploaded.drop_duplicates()
    
    # Convert the exploded column "skills" to lowercase
    exploded_df_lower = convert_to_lower(exploded_df1,"skillS")
    
    # Dropping the skills which have numbers and not words
    duplicate1 = exploded_df1.groupBy("skills").count().filter(col("skills") > 1)

    # Get the values to remove as a list
    values_to_remove = [row[0] for row in duplicate1.select("skills").collect()]

    # Remove values from the DataFrame that are in values_to_remove
    exploded_df_lower = exploded_df_lower.filter(~col("skills").isin(values_to_remove))
    
    # Group the DataFrame by skill and count the frequency of each skill
    exploded_df_lower = exploded_df_lower.groupBy("skills").agg(count("skills").alias("Frequency"))

    # Display the top 10 distinct skills in lowercase and their frequencies
    exploded_df_lower.orderBy("Frequency", ascending=False).show(num_top_skills, truncate=False)
    
# Funtion Call
distinct_skills_lowercase(main_dataframe, 10)

+--------------------+---------+
|skills              |Frequency|
+--------------------+---------+
|java                |2759     |
|javascript          |2738     |
|sales               |2680     |
|business development|2108     |
|marketing           |1809     |
|sql                 |1564     |
|jquery              |1547     |
|html                |1539     |
|communication skills|1537     |
|bpo                 |1530     |
+--------------------+---------+
only showing top 10 rows



In [0]:
#2. FURTHER EXPLORATION QUESTION 4

from pyspark.sql.functions import lower

# Define a function convert column in dataframe to lowercase
def convert_to_lower(df, column):
    """
    convert a column in a datframe to lowercase
    Parameters:
        df: PySpark DataFrame to modify
        column: column name to convert to lowercase
    Returns:
        dataframe and column name which is converted to lowercase
    """
    return df.withColumn(column, lower(df[column]))

# Define a function count distinct lowercase skills 
def distinct_skills_lowercase(df, num_top_skills):
    """
    Counts the frequency of individual skills in a PySpark DataFrame
    Parameters:
        df: PySpark DataFrame to modify
        num_top_skills (int): The number of top skills to return
    Returns:
        None
    """
    #remove duplicates values
    exploded_df1 = new_exploaded.drop_duplicates()
    
    # Convert the exploded column "skills" to lowercase
    exploded_df_lower = convert_to_lower(exploded_df1,"skillS")
    
    # Dropping the skills which have numbers and not words
    duplicate1 = exploded_df1.groupBy("skills").count().filter(col("skills") > 1)

    # Get the values to remove as a list
    values_to_remove = [row[0] for row in duplicate1.select("skills").collect()]

    # Remove values from the DataFrame that are in values_to_remove
    exploded_df_lower = exploded_df_lower.filter(~col("skills").isin(values_to_remove))
    
    # Group the DataFrame by skill and count the frequency of each skill
    exploded_df_lower = exploded_df_lower.groupBy("skills").agg(count("skills").alias("Frequency"))

    # Filtering top 10 distinct skills in lowercase and their frequencies
    exploded_df_lower = exploded_df_lower.orderBy("Frequency", ascending=False).limit(num_top_skills)
    
    # Visualisation of the result
    display(exploded_df_lower)
    
# Funtion Call
distinct_skills_lowercase(main_dataframe, 10)



skills,Frequency
java,2759
javascript,2738
sales,2680
business development,2108
marketing,1809
sql,1564
jquery,1547
html,1539
communication skills,1537
bpo,1530


Databricks visualization. Run in Databricks to view.

In [0]:
# QUESTION 5

# Define a function convert column in dataframe to lowercase
def convert_to_lower(df, column):
    """
    convert a column in a datframe to lowercase
    
    Parameters:
        df: PySpark DataFrame to modify
        column: column name to convert to lowercase

    Returns:
        dataframe and column name which is converted to lowercase
    """
    return df.withColumn(column, lower(df[column]))

# Define a function join two dataframe and count the skills
def joined_skill(df):
    """
    Counts the frequency of skills and join two dataframe
    Parameters:
        df: PySpark DataFrame to modify
    Returns:
        None
    """
    
    # Convert Example column to lowercase in o_net DataFrame
    df2 = convert_to_lower(o_net, "Example")
    
    # Removing duplicate values from o_net dataset
    df2 = df2.drop_duplicates()
    
    #removing duplicate values from the exploaded_df job description dataset
    exploded_df1=exploded_df.drop_duplicates()

    # Convert the exploded column "skills" to lowercase
    df1 = convert_to_lower(exploded_df1,"skill")

    # Join the two DataFrames on the lowercase skill and Example columns
    joined_df = df1.join(df2, df1.skill == df2.Example)
    
    #Change in the number of skills before and after the join
    a = df1.count()
    b = joined_df.count()
    change = b-a

    # Print the count of skills before join
    print("Skills Count Before Join: {0:,.0f}".format(df1.count()))
    
    # Print the count of skills before after the join
    print("Skills Count After Join: {0:,.0f}".format(joined_df.count()))
    
    # Print the change in number of skills before and after the join
    print("Change in the number of skills before and after the join: {0:,.0f}".format(change))

# call funtion joined_skill
joined_skill(main_dataframe)


Skills Count Before Join: 463,803
Skills Count After Join: 1,101,498
Change in the number of skills before and after the join: 637,695


In [0]:
# 1. FURTHER EXPLORATION FOR QUESTION 5

# Define a function convert column in dataframe to lowercase
def convert_to_lower(df, column):
    """
    convert a column in a datframe to lowercase
    
    Parameters:
        df: PySpark DataFrame to modify
        column: column name to convert to lowercase

    Returns:
        dataframe and column name which is converted to lowercase
    """
    return df.withColumn(column, lower(df[column]))

# Define a function join two dataframe and count the skills
def joined_skill(df):
    """
    Counts the frequency of skills and join two dataframe
    Parameters:
        df: PySpark DataFrame to modify
    Returns:
        None
    """
    
    # Convert Example column to lowercase in o_net DataFrame
    df2 = convert_to_lower(o_net, "Example")
    
    # Removing duplicate values from o_net dataset
    df2 = df2.drop_duplicates()
    
    #removing duplicate values from the exploaded_df job description dataset
    exploded_df1=exploded_df.drop_duplicates()

    # Convert the exploded column "skills" to lowercase
    df1 = convert_to_lower(exploded_df1,"skill")

    # Join the two DataFrames on the lowercase skill and Example columns
    joined_df = df1.join(df2, df1.skill == df2.Example)
    
    #Change in the number of skills before and after the join
    a = df1.count()
    b = joined_df.count()
    change = b-a

    data = [('Skills Before Join', a),('Skills After Join', b)]
    df_1 = spark.createDataFrame(data, ['Skills', 'Count'])
    
    # Print the count of skills before join
    #print("Skills Count Before Join: {0:,.0f}".format(df1.count()))
    
    # Print the count of skills before after the join
    #print("Skills Count After Join: {0:,.0f}".format(joined_df.count()))
    
    # Print the change in number of skills before and after the join
    #print("Change in the number of skills before and after the join: {0:,.0f}".format(change))
    
    display(df_1)

# call funtion joined_skill
joined_skill(main_dataframe)


Skills,Count
Skills Before Join,463803
Skills After Join,1101498


Databricks visualization. Run in Databricks to view.

In [0]:
# 2. FURTHER EXPLORATION FOR QUESTION 5

# Define a function convert column in dataframe to lowercase
def convert_to_lower(df, column):
    """
    convert a column in a datframe to lowercase
    
    Parameters:
        df: PySpark DataFrame to modify
        column: column name to convert to lowercase

    Returns:
        dataframe and column name which is converted to lowercase
    """
    return df.withColumn(column, lower(df[column]))

# Define a function join two dataframe and count the skills
def joined_skill_01():
    """
    Counts the frequency of skills and join two dataframe
    Parameters:
        df: PySpark DataFrame to modify
    Returns:
        None
    """
    
    # Convert Example column to lowercase in o_net DataFrame
    df2 = convert_to_lower(o_net, "Example")
    
    # Removing duplicate values from dataset
    df2 = df2.drop_duplicates()
    
    # Removing duplicate values from dataset which have no stop words in skill column
    exploded_df1 = new_exploaded.drop_duplicates()

    # Convert the exploded column "skills" to lowercase
    df1 = convert_to_lower(exploded_df1,"skills")
    
    # Calculating the values in skills column which are numbers
    duplicate1 = df1.groupBy("skills").count().filter(col("skills") > 1)
    
    # Calculating the values in Example column which are numbers
    duplicate2 = df2.groupBy("Example").count().filter(col("Example") > 1)
    
    # Get the skills values to remove as a list
    values_to_remove = [row[0] for row in duplicate1.select("skills").collect()]
    
    # Get the Example values to remove as a list
    values_to_remove2 = [row[0] for row in duplicate2.select("Example").collect()]

    # Remove values from the DataFrame that are in values_to_remove
    df1 = df1.filter(~col("skills").isin(values_to_remove))
    
    # Remove values from the DataFrame that are in values_to_remove
    df2 = df2.filter(~col("Example").isin(values_to_remove))

    # Join the two DataFrames on the lowercase skill and Example columns
    joined_df = df2.join(df1, df2.Example == df1.skills)
    
    #Change in the number of skills before and after the join
    a = df1.count()
    b = joined_df.count()
    change = b-a

    # Print the count of skills before join
    # SKILLS NOT HAVING ANY NUMERIC VALUE AND STOPWORDS)
    print("Skills Count Before Join: {0:,.0f}".format(df1.count()))
    
    # Print the count of skills before after the join
    print("Skills Count After Join: {0:,.0f}".format(joined_df.count()))
    
    # Print the change in number of skills before and after the join
    print("Change in the number of skills before and after the join: {0:,.0f}".format(change))

# call funtion joined_skill
joined_skill_01()


Skills Count Before Join: 462,544
Skills Count After Join: 1,101,498
Change in the number of skills before and after the join: 638,954


In [0]:
# QUESTION 6

# Define a function convert column in dataframe to lowercase
def convert_to_lower(df, column):
    """
    convert a column in a datframe to lowercase
    
    Parameters:
        df: PySpark DataFrame to modify
        column: column name to convert to lowercase

    Returns:
        dataframe and column name which is converted to lowercase
    """
    return df.withColumn(column, lower(df[column]))

def commodity_title(df, onet_df, num_of_rows):
    
    """
    convert a column in a datframe to lowercase
    
    Parameters:
        df: PySpark DataFrame to modify
        onet_df: PySpark DataFrame to modify
        num_of_rows: The number of top skills to return

    Returns:
        None
    """
    #Convert Example column to lowercase in onet_df
    onet_df = convert_to_lower(onet_df,"Example")

    #remove the duplicate values
    exploded_df1=exploded_df.drop_duplicates()
    
    # Converting exploded column skills into lowercase
    exploded_df_lower = convert_to_lower(exploded_df1,"skill")

    # Join the DataFrames on skill and Example columns
    joined_df = exploded_df_lower.join(onet_df, exploded_df_lower.skill == onet_df.Example)

    # Group by Commodity Title and count the frequency of each title
    commodity_df = joined_df.groupBy("Commodity Title").count() \
        .sort('count', ascending=False) \
        .withColumnRenamed("count", "Frequency")

    # Display the top 10 commodities
    commodity_df.show(num_of_rows, truncate=False)

# call commodity_title function
# Skill DataFrame is called `lower_skills_jd` and the o_net DataFrame is called `o_net`
commodity_title(main_dataframe, o_net, 10)

+-------------------------------------------------+---------+
|Commodity Title                                  |Frequency|
+-------------------------------------------------+---------+
|Object or component oriented development software|324521   |
|Web platform development software                |298754   |
|Operating system software                        |190926   |
|Development environment software                 |53013    |
|Data base management system software             |44132    |
|Analytical or scientific software                |33552    |
|Web page creation and editing software           |31682    |
|Data base user interface and query software      |29436    |
|Spreadsheet software                             |18568    |
|File versioning software                         |13846    |
+-------------------------------------------------+---------+
only showing top 10 rows



In [0]:
# 1. FURTHER EXPLORATION FOR QUESTION 6

# Define a function convert column in dataframe to lowercase
def convert_to_lower(df, column):
    """
    convert a column in a datframe to lowercase
    
    Parameters:
        df: PySpark DataFrame to modify
        column: column name to convert to lowercase

    Returns:
        dataframe and column name which is converted to lowercase
    """
    return df.withColumn(column, lower(df[column]))

def commodity_title(df, onet_df, num_of_rows):
    
    """
    convert a column in a datframe to lowercase
    
    Parameters:
        df: PySpark DataFrame to modify
        onet_df: PySpark DataFrame to modify
        num_of_rows: The number of top skills to return

    Returns:
        None
    """
    #Convert Example column to lowercase in onet_df
    onet_df = convert_to_lower(onet_df,"Example")

    #remove the duplicate values
    exploded_df1=exploded_df.drop_duplicates()
    
    # Converting exploded column skills into lowercase
    exploded_df_lower = convert_to_lower(exploded_df1,"skill")

    # Join the DataFrames on skill and Example columns
    joined_df = exploded_df_lower.join(onet_df, exploded_df_lower.skill == onet_df.Example)

    # Group by Commodity Title and count the frequency of each title
    commodity_df = joined_df.groupBy("Commodity Title").count() \
        .sort('count', ascending=False) \
        .withColumnRenamed("count", "Frequency").limit(num_of_rows)

    # Display the top 10 commodities
    display(commodity_df)

# call commodity_title function
# Skill DataFrame is called `lower_skills_jd` and the o_net DataFrame is called `o_net`
commodity_title(main_dataframe, o_net, 10)

Commodity Title,Frequency
Object or component oriented development software,324521
Web platform development software,298754
Operating system software,190926
Development environment software,53013
Data base management system software,44132
Analytical or scientific software,33552
Web page creation and editing software,31682
Data base user interface and query software,29436
Spreadsheet software,18568
File versioning software,13846


Databricks visualization. Run in Databricks to view.